# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core import Model 
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.compute import ComputeTarget, AmlCompute
from pprint import pprint 
import joblib
import os

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
#ANCLA:una vez que ya fue leida la data enviar un mensaje de realizado
#ANCLA: enviar una salida de datos del dataframe para que pueda visualizarse
#getting data: the dataset was uploaded to Azure Environment but can be accesible through an online link for example
from azureml.data.dataset_factory import TabularDatasetFactory

rawdata_homic2018 = "https://raw.githubusercontent.com/hftamayo/azuremlproj03/main/cad2018.csv"
dshomic2018 = TabularDatasetFactory.from_delimited_files(path=rawdata_homic2018, separator=',')
pdhomic2018 = dshomic2018.to_pandas_dataframe().dropna()

In [3]:
#ANCLA: hacer un check en caso que el experimento ya exista, enviar un mensaje que fue creado o que ya existe segun sea el caso
#setting up workspace and dataset
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'exp_p03_ht'

experiment=Experiment(ws, experiment_name)

In [4]:
# setting up IaaS where the experiment will run on it
ccname = "ccp03aml"
try:
    ccluster = ComputeTarget(ws, ccname)
    print(f"{ccname} exists or is in use!, choose a different name")
except:
    ccconfig = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=5)
    ccluster = ComputeTarget.create(ws, ccname, ccconfig)
ccluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [5]:
#ANCLA: enviar un mensaje que las configs fueron seteadas
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
        task='classification',
        compute_target=ccluster,
        training_data=dshomic2018,
        label_column_name='sexo',
        n_cross_validations=5,
        **automl_settings
)

In [6]:
# TODO: Submit your experiment
exprun = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
RunDetails(exprun).show()
exprun.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of 

        60   TruncatedSVDWrapper XGBoostClassifier          0:00:43       0.8790    0.8817
        61   StandardScalerWrapper XGBoostClassifier        0:00:45       0.8802    0.8817
        62   SparseNormalizer LightGBM                      0:00:47       0.8790    0.8817
        63   SparseNormalizer LightGBM                      0:00:46       0.8790    0.8817
        64   StandardScalerWrapper LightGBM                 0:00:46       0.8790    0.8817
        65   SparseNormalizer XGBoostClassifier             0:00:46       0.8790    0.8817
        66   SparseNormalizer LightGBM                      0:00:41       0.8790    0.8817
        67   MaxAbsScaler LightGBM                          0:00:44       0.8790    0.8817
        68   MaxAbsScaler LightGBM                          0:00:46       0.8790    0.8817
        69   StandardScalerWrapper XGBoostClassifier        0:01:01       0.8784    0.8817
        70   SparseNormalizer XGBoostClassifier             0:00:47       0.8790    0.8817

{'runId': 'AutoML_930812e7-f3ff-41b3-9a51-bb7eb2d5d514',
 'target': 'ccp03aml',
 'status': 'Completed',
 'startTimeUtc': '2021-01-23T17:04:00.407123Z',
 'endTimeUtc': '2021-01-23T17:35:41.027496Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'ccp03aml',
  'AMLSettingsJsonString': '{"path":null,"name":"exp_p03_ht","subscription_id":"aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee","resource_group":"aml-quickstarts-135796","workspace_name":"quick-starts-ws-135796","region":"southcentralus","compute_target":"ccp03aml","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes":null,"featurization

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [9]:
#import azureml.train.automl
# obtaining best run and fitted model
brmodel, fmodel = exprun.get_output()

# Print the best run
print(brmodel)

# Get all metrics of the best run model
brmetrics = brmodel.get_metrics()

# Print all metrics of the best run model
for metric_name in brmetrics:
    metric = brmetrics[metric_name]
    print(metric_name, metric)
    

Package:azureml-automl-runtime, training version:1.20.0, current version:1.19.0
Package:azureml-core, training version:1.20.0, current version:1.19.0
Package:azureml-dataprep, training version:2.7.2, current version:2.6.1
Package:azureml-dataprep-native, training version:27.0.0, current version:26.0.0
Package:azureml-dataprep-rslex, training version:1.5.0, current version:1.4.0
Package:azureml-dataset-runtime, training version:1.20.0, current version:1.19.0.post1
Package:azureml-defaults, training version:1.20.0, current version:1.19.0
Package:azureml-interpret, training version:1.20.0, current version:1.19.0
Package:azureml-pipeline-core, training version:1.20.0, current version:1.19.0
Package:azureml-telemetry, training version:1.20.0, current version:1.19.0
Package:azureml-train-automl-client, training version:1.20.0, current version:1.19.0
Package:azureml-train-automl-runtime, training version:1.20.0, current version:1.19.0


Run(Experiment: exp_p03_ht,
Id: AutoML_930812e7-f3ff-41b3-9a51-bb7eb2d5d514_74,
Type: azureml.scriptrun,
Status: Completed)
accuracy 0.8822466144613257
precision_score_macro 0.8711656517145284
recall_score_macro 0.5160587250315347
AUC_micro 0.9074477041670406
average_precision_score_macro 0.5736527709467525
norm_macro_recall 0.0321174500630693
recall_score_micro 0.8822466144613257
precision_score_micro 0.8822466144613257
log_loss 0.3560024970915745
AUC_weighted 0.6342404456233657
f1_score_micro 0.8822466144613257
precision_score_weighted 0.8795672390692376
average_precision_score_micro 0.8916447016411132
matthews_correlation 0.1470038086296908
weighted_accuracy 0.9812711363162606
average_precision_score_weighted 0.8342450189707117
f1_score_macro 0.4997857299780145
AUC_macro 0.6342404456233657
recall_score_weighted 0.8822466144613257
f1_score_weighted 0.8311503055843815
balanced_accuracy 0.5160587250315347
confusion_matrix aml://artifactId/ExperimentRun/dcid.AutoML_930812e7-f3ff-41b3-9a

In [13]:
#TODO: Save the best model
autobmlp3ht = brmodel.register_model(model_path='outputs/model.pkl', model_name='p3automl_ht',
                        tags={'Method of execution':'Auto ML'},
                        properties={'Accuracy': brmetrics['accuracy']})

print(autobmlp3ht)

Model(workspace=Workspace.create(name='quick-starts-ws-135796', subscription_id='aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee', resource_group='aml-quickstarts-135796'), name=p3automl_ht, id=p3automl_ht:2, version=2, tags={'Method of execution': 'Auto ML'}, properties={'Accuracy': '0.8822466144613257'})


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [11]:
# Download scoring file 
brmodel.download_file('outputs/scoring_file_v_1_0_0.py', 'scoring.py')

# Download environment file
brmodel.download_file('outputs/conda_env_v_1_0_0.yml', 'enviro.yml')

In [14]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

infcfg = InferenceConfig(entry_script='scoring.py',
                                    environment=Environment.from_conda_specification(name='myenv',file_path='enviro.yml'))

# deploying the model via WebService
from azureml.core.webservice import AciWebservice

depcfg = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
webservice = Model.deploy(ws, "wsp03ht01", [autobmlp3ht], infcfg, depcfg)
webservice.wait_for_deployment(show_output = True)
print(webservice.state)

print(webservice.scoring_uri)

print(webservice.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running........................................................................................................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://c0bc6d8a-98e8-4d7d-b847-1fbbb5c3303a.southcentralus.azurecontainer.io/score
http://c0bc6d8a-98e8-4d7d-b847-1fbbb5c3303a.southcentralus.azurecontainer.io/swagger.json


In [15]:
import json

#Importing the data set for testing 
tsthomic2018 = pdhomic2018.sample(5) 
lblhomic2018 = tsthomic2018.pop('sexo')

tstdatahomic2018 = json.dumps({'pdhomic2018': tsthomic2018.to_dict(orient='records')})

print(tstdatahomic2018)

{"pdhomic2018": [{"id": 2231, "regfalle": 19189, "edad": 36, "deptoocuhe": 6, "tipoarma": 1, "pracaut": 1}, {"id": 1173, "regfalle": 20935, "edad": 25, "deptoocuhe": 6, "tipoarma": 1, "pracaut": 0}, {"id": 2645, "regfalle": 19696, "edad": 23, "deptoocuhe": 9, "tipoarma": 1, "pracaut": 1}, {"id": 1203, "regfalle": 16095, "edad": 17, "deptoocuhe": 6, "tipoarma": 1, "pracaut": 0}, {"id": 624, "regfalle": 21025, "edad": 22, "deptoocuhe": 11, "tipoarma": 10, "pracaut": 1}]}


TODO: In the cell below, send a request to the web service you deployed to test it.

In [16]:
import requests

headers = {'Content-type': 'application/json'}

response = requests.post(webservice.scoring_uri, tstdatahomic2018, headers=headers)

In [23]:
# Print results from the inference
print(response.text)

list index out of range


In [19]:
# Print original labels
print(lblhomic2018)

2230    0
1172    0
2644    0
1202    0
623     0
Name: sexo, dtype: int64


TODO: In the cell below, print the logs of the web service and delete the service

In [20]:
print(webservice.get_logs())

2021-01-23T18:05:46,779691626+00:00 - nginx/run 
2021-01-23T18:05:46,779891436+00:00 - rsyslog/run 
2021-01-23T18:05:46,780430861+00:00 - gunicorn/run 
2021-01-23T18:05:46,780987987+00:00 - iot-server/run 
/usr/sbin/nginx: /azureml-envs/azureml_fc7e1840225416a471556263041ca33e/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_fc7e1840225416a471556263041ca33e/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_fc7e1840225416a471556263041ca33e/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_fc7e1840225416a471556263041ca33e/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_fc7e1840225416a471556263041ca33e/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

In [ ]:
webservice.delete()